# 如何创建自定义工具 How to create custom 

构建代理时，您需要为其提供可以使用的工具列表。除了实际调用的函数外，工具还包含几个组件：

| 属性       | 类型               | 描述                                                                                                                  |
|------------|--------------------|-----------------------------------------------------------------------------------------------------------------------|
| name       | str                | 必须唯一 -- 在提供给LLM或代理的一组工具中。                                                                             |
| description| str                | 描述工具的功能。作为LLM或代理的上下文使用。                                                                           |
| args_schema| Pydantic BaseModel | 可选但推荐，可用于提供更多信息（例如，少量示例）或对预期参数进行验证。                                                 |
| return_direct| boolean          | 仅对代理相关。当为True时，在调用了给定的工具后，代理将停止并将结果直接返回给用户。                                      |

LangChain提供了创建工具的三种方式：

1. 使用`@tool`装饰器 —— 定义自定义工具的最简单方式。
2. 使用`StructuredTool.from_function`类方法 —— 这与`@tool`装饰器相似，但允许更多的配置，并且可以指定同步和异步实现。
3. 通过继承自`BaseTool` —— 这是最灵活的方法，它提供了最大程度的控制权，但代价是需要更多的努力和代码编写。

对于大多数应用场景，使用`@tool`装饰器或`StructuredTool.from_function`类方法应该已经足够。

## @tool decorator 装饰器
此 @tool 装饰器是定义自定义工具的最简单方法。  
装饰器默认使用函数名称作为工具名称，但可以通过传递字符串作为第一个参数来覆盖此名称。  
此外，装饰器将使用函数的文档字符串作为工具的描述 - 因此必须提供文档字符串。

In [2]:
from langchain_core.tools import tool

@tool
def multiply(a:int , b:int) -> int:
    """multiply 2 nums"""
    return a * b


#让我们检查一下与该工具相关的一些属性。
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply 2 nums
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


或者创建一个异步实现，如下所示：

In [4]:
from langchain_core.tools import tool

@tool
async def amultiply(a:int , b:int) -> int:
    """async multiply 2 nums"""
    return a * b 

print(amultiply.name)
print(amultiply.description)
print(amultiply.args)

amultiply
async multiply 2 nums
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


您还可以通过将工具名称和 JSON 参数传递到工具装饰器来自定义它们。

In [8]:
from langchain.pydantic_v1 import BaseModel,Field

class CalculatorInput(BaseModel):
    a: int = Field(description="first num")
    b: int = Field(description="second num")

@tool("multiplication-tool",args_schema=CalculatorInput,return_direct=True)
def multiply(a:int,b:int) -> int:
    """multiply 2 num by extends BaseModel"""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
multiply 2 num by extends BaseModel
{'a': {'title': 'A', 'description': 'first num', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second num', 'type': 'integer'}}
True


## StructuredTool
StructuredTool.from_function 类方法比 @tool 装饰器提供了更多的可配置性，而不需要太多额外的代码。

In [12]:
from langchain_core.tools import StructuredTool

def mul(a:int,b:int) -> int:
    """mul two nums."""
    return a * b

async def amul(a:int,b:int) -> int:
    """async mul two nums."""
    return a * b

calc = StructuredTool.from_function(func=mul,coroutine=amul)

print(calc.invoke({"a":2,"b":3}))
print(await calc.ainvoke({"a":2,"b":5}))

6
10


In [13]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

print(calculator.invoke({"a": 2, "b": 3}))
print(calculator.name)
print(calculator.description)
print(calculator.args)

6
Calculator
multiply numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


## 子类化 BaseTool
您可以通过从 BaseTool 子类化来定义自定义工具。这可以最大程度地控制工具定义，但需要编写更多代码。

In [15]:
from typing import Optional,Type

from langchain.pydantic_v1 import BaseModel
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun
)
from langchain_core.tools import BaseTool

class CalcInput(BaseModel):
    a: int = Field(description="1st num")
    b: int = Field(description="2nd num")

class CustomCalcTool(BaseTool):
    name = "Calc"
    description="当你需要回答数学问题时很有用"
    args_schema:Type[BaseModel] = CalcInput
    return_direct: bool = True

    def _run(self,a:int,b:int,run_manager:Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return a * b

    async def _arun(self,a:int,b:int,run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool async"""
        return self._run(a,b,run_manager=run_manager.get_sync())


In [21]:
mul = CustomCalcTool()
print(mul.name)
print(mul.description)
print(mul.args)
print(mul.return_direct)

print(mul.invoke({"a":"2","b":"3"}))
print(await mul.ainvoke({"a":"5","b":"3"}))

Calc
当你需要回答数学问题时很有用
{'a': {'title': 'A', 'description': '1st num', 'type': 'integer'}, 'b': {'title': 'B', 'description': '2nd num', 'type': 'integer'}}
True
6
15


## 如何创建异步工具
所有 Runnable 都公开了invoke 和 ainvoke 方法（以及其他方法，如 batch、abatch、astream 等）。

因此，即使您只提供工具的同步实现，您仍然可以使用 ainvoke 接口，但有一些重要的事情需要了解：

 - 默认情况下，LangChain 提供了一个异步实现，该实现假定该函数的计算成本很高，因此它将执行委托给另一个线程。
 - 如果您在异步代码库中工作，则应该创建异步工具而不是同步工具，以避免因该线程而产生少量开销。
 - 如果您同时需要同步和异步实现，请使用 StructuredTool.from_function 或 BaseTool 的子类。
 - 如果同时实现同步和异步，并且同步代码运行速度很快，请覆盖默认的 LangChain 异步实现并简单地调用同步代码。
 - 您不能也不应该将同步调用与异步工具一起使用。

In [22]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(
    await calculator.ainvoke({"a": 2, "b": 5})
)  # 使用默认的 LangChain 异步实现会产生少量开销

6
10


In [23]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)

print(calculator.invoke({"a": 2, "b": 3}))
print(
    await calculator.ainvoke({"a": 2, "b": 5})
)  # 使用提供的 amultiply 无需额外开销

6
10


当仅提供异步定义时，您不应该也不能使用 .invoke。

In [24]:
@tool
async def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


try:
    multiply.invoke({"a": 2, "b": 3})
except NotImplementedError:
    print("Raised not implemented error. You should not be doing this.")

Raised not implemented error. You should not be doing this.


## 处理工具错误
如果您使用带有代理的工具，则可能需要错误处理策略，以便代理可以从错误中恢复并继续执行。

一个简单的策略是从工具内部抛出 ToolException 并使用 handle_tool_error 指定错误处理程序。

指定错误处理程序后，将捕获异常，错误处理程序将决定从工具返回哪个输出。

您可以将 handle_tool_error 设置为 True、字符串值或函数。如果它是一个函数，则该函数应将 ToolException 作为参数并返回一个值。

请注意，仅引发 ToolException 是无效的。您需要首先设置工具的 handle_tool_error，因为它的默认值为 False。

In [25]:
from langchain_core.tools import ToolException


def get_weather(city: str) -> int:
    """Get weather for the given city."""
    raise ToolException(f"Error: There is no city by the name of {city}.")

这是一个具有默认 handle_tool_error=True 行为的示例。

In [26]:
get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=True,
)

get_weather_tool.invoke({"city": "foobar"})

'Error: There is no city by the name of foobar.'

我们可以将handle_tool_error设置为一个始终返回的字符串。

In [27]:
get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error="There is no such city, but it's probably above 0K there!",
)

get_weather_tool.invoke({"city": "foobar"})

"There is no such city, but it's probably above 0K there!"

使用函数处理错误：

In [28]:
def _handle_error(error: ToolException) -> str:
    return f"The following errors occurred during tool execution: `{error.args[0]}`"


get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=_handle_error,
)

get_weather_tool.invoke({"city": "foobar"})

'The following errors occurred during tool execution: `Error: There is no city by the name of foobar.`'